In [1]:
import numpy as np
from collections import UserList

In [2]:
class AutoCorrList(UserList):

    def __init__(self, initlist=None):
        if initlist == None:
            super().__init__()
        elif type(initlist[0]) == type(np.ndarray([])):
            super().__init__(initlist)
        else:
            raise ValueError(f"need to data type = numpy.ndarray: actual {type(initlist[0])}")

    def __getitem__(self, i):
        if isinstance(i, slice):
            raise NotImplementedError("accessor for AutoCorrList")
            # return self.__class__(self.data[i])
        else:
            if i < 0:
                return self.data[-i].T
            return self.data[i]

    def __setitem__(self, i, item):
        if i < 0:
            raise ValueError(f"need index >= 0: actual {i}")
        self.data[i] = item

In [3]:
d = 2
p = 4
mat = [np.random.randn(d, d) for i in range(0, p+1)]
autocorrs = AutoCorrList(mat)
coeffs = [np.random.randn(d, d) for i in range(1, p+1)]

In [4]:
e = np.eye(d, d)


def Phi(coeffs, k):
    ret = autocorrs[k]
    for l in range(1, p+1):
        ret = ret + coeffs[l-1] @ autocorrs[k-l]
    return ret


def grad_elem(coeffs, l, i, j):
    assert l > 0
    ret = np.zeros_like(autocorrs[0])
    for k in range(1, p+1):
        ret = ret + autocorrs[k-l] @ (Phi(coeffs, k).T)
    return e[j].T @ ret @ e[i]


def gradient(coeffs):
    return [np.array([[grad_elem(coeffs, l, i, j) for j in range(d)] for i in range(d)]) for l in range(1, p+1)]


def loss(coeffs):
    ret = 0.0
    for l in range(1,p+1):
        P = Phi(coeffs, l)
        ret += np.trace(P.T @ P)
    return ret

In [5]:
iters = 10000
lr = 0.1
it_per_epoch = iters / 500

coeffs = [np.random.randn(d, d) for i in range(1, p+1)]
# coeffs = [expect]
print(coeffs)
print(f"init loss: {loss(coeffs)}")

for i in range(iters):
    grads = gradient(coeffs)
    for l in range(1, p+1):
        coeffs[l-1] -= lr * grads[l-1]
        
    if i % it_per_epoch == 0:
        print(f"loss at iter #{i}: {loss(coeffs)}")

coeffs

[array([[ 0.07378134, -0.76014759],
       [ 0.62124745, -0.49926742]]), array([[ 1.48165617, -0.22034993],
       [ 1.18795126, -0.11645063]]), array([[ 0.04004554,  0.72325538],
       [-1.20996665,  0.92684398]]), array([[0.78117998, 2.61641606],
       [1.60122119, 0.50385444]])]
init loss: 117.13031152923107
loss at iter #0: 73.79823302679787
loss at iter #500: 4.7561866475353016e+67
loss at iter #1000: 3.681661571104213e+133
loss at iter #1500: 2.8498948692792993e+199
loss at iter #2000: 2.2060421929298746e+265


/var/folders/22/z13bk7z94l5634tn753b47kh0000gn/T/ipykernel_77584/1151860656.py:27: RuntimeWarning: overflow encountered in matmul
  ret += np.trace(P.T @ P)


loss at iter #2500: inf
loss at iter #3000: inf
loss at iter #3500: inf
loss at iter #4000: inf
loss at iter #4500: inf


/var/folders/22/z13bk7z94l5634tn753b47kh0000gn/T/ipykernel_77584/1151860656.py:15: RuntimeWarning: overflow encountered in add
  ret = ret + autocorrs[k-l] @ (Phi(coeffs, k).T)
/var/folders/22/z13bk7z94l5634tn753b47kh0000gn/T/ipykernel_77584/1151860656.py:16: RuntimeWarning: invalid value encountered in matmul
  return e[j].T @ ret @ e[i]


loss at iter #5000: nan
loss at iter #5500: nan
loss at iter #6000: nan
loss at iter #6500: nan
loss at iter #7000: nan
loss at iter #7500: nan
loss at iter #8000: nan
loss at iter #8500: nan
loss at iter #9000: nan
loss at iter #9500: nan


[array([[nan, nan],
        [nan, nan]]),
 array([[nan, nan],
        [nan, nan]]),
 array([[nan, nan],
        [nan, nan]]),
 array([[nan, nan],
        [nan, nan]])]

In [6]:
loss(coeffs)

nan